# 3. Hafta Lab-2: $L1$ ve $L2$ Regularization Sonuçlarının İncelenmesi
## BGM 565: Siber Güvenlik için Makine Öğrenme Yöntemleri
## İstanbul Şehir Üni. - Bilgi Güvenliği Müh.
### Dr. Ferhat Özgür Çatak
Bu lab çalışmasında lojistik regresyon algoritmasında $L2$ regularization yönteminde $\lambda$ değeriyle ağırlık vektörünün $\mathbf{w}$ değişimi KDDCUP'99 veri kümesinde uygulayacağız.

In [1]:
import numpy as np
import os
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error

In [2]:
# veri kumesini oku
kolon_adlari = ['duration','protocol_type','service','flag','src_bytes','dst_bytes','land','wrong_fragment','urgent',
'hot','num_failed_logins','logged_in','num_compromised','root_shell','su_attempted','num_root','num_file_creations',
'num_shells','num_access_files','num_outbound_cmds','is_host_login','is_guest_login','count','srv_count',
'serror_rate','srv_serror_rate','rerror_rate','srv_rerror_rate','same_srv_rate','diff_srv_rate','srv_diff_host_rate',
'dst_host_count','dst_host_srv_count','dst_host_same_srv_rate','dst_host_diff_srv_rate','dst_host_same_src_port_rate',
'dst_host_srv_diff_host_rate','dst_host_serror_rate','dst_host_srv_serror_rate','dst_host_rerror_rate',
'dst_host_srv_rerror_rate','label']

verikumesi = pd.read_csv("kddcup99.tar.gz",compression="gzip", names=kolon_adlari, 
low_memory=False, skiprows=1)

# ilgili kolonlari sec
secilecek_kolonlar = ['serror_rate','srv_serror_rate',
'rerror_rate','srv_rerror_rate','same_srv_rate','diff_srv_rate','srv_diff_host_rate','dst_host_count',
'dst_host_srv_count','dst_host_same_srv_rate','dst_host_diff_srv_rate','dst_host_same_src_port_rate',
'dst_host_srv_diff_host_rate','dst_host_serror_rate','dst_host_srv_serror_rate','dst_host_rerror_rate',
'dst_host_srv_rerror_rate']


X = verikumesi[secilecek_kolonlar].as_matrix()
y = verikumesi['label'].apply(lambda d:0 if d == 'normal.' else 1).as_matrix()

Veri kumesini egitim ve test veri kumesi olarak ayır

In [3]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

$\lambda$ değerlerini tanımla

In [4]:
lambdas = [10**-15, 10**-10, 10**-8, 0.0001, 0.001, 0.01, 1, 5, 10, 20, 50, 100, 1000]

Sonuçları kaydetmek için pandas dataframe tanımla.

In [5]:
col = ['lambda','MSE_Train','MSE_Test','w_0'] + ['w_%d'%i for i in range(1,X.shape[1]+1)] 
ind = [i for i in range(1,len(lambdas)+1)]
coef_matrix_simple = pd.DataFrame(index=ind, columns=col)
coef_matrix_simple

,lambda,MSE_Train,MSE_Test,w_0,w_1,w_2,w_3,w_4,w_5,w_6,...,w_8,w_9,w_10,w_11,w_12,w_13,w_14,w_15,w_16,w_17
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Her bir $\lambda$ değeri için $L2$ regularization kullanarak *LogisticRegression* algoritması kullanılarak sınıflandırma modelini olustur.

In [6]:
i = 0
for l in lambdas:
    clf = LogisticRegression(penalty='l2', C=1/l)
    clf.fit(X_train,y_train)
    
    y_hat_train = clf.predict(X_train)
    mse_train = mean_squared_error(y_train, y_hat_train)
    
    y_hat_test = clf.predict(X_test)
    mse_test = mean_squared_error(y_test, y_hat_test)
    
    coef_matrix_simple.iloc[i,0] = l
    coef_matrix_simple.iloc[i,1] = mse_train
    coef_matrix_simple.iloc[i,2] = mse_test
    coef_matrix_simple.iloc[i,3] = clf.intercept_[0]
    coef_matrix_simple.iloc[i,4:] = clf.coef_
    i += 1
    print('Tamamlanan : % {0}'.format(i/len(lambdas)*100), end = '\r',flush=False)
    #os.system('cls' if os.name=='nt' else 'clear')
coef_matrix_simple

,lambda,MSE_Train,MSE_Test,w_0,w_1,w_2,w_3,w_4,w_5,w_6,...,w_8,w_9,w_10,w_11,w_12,w_13,w_14,w_15,w_16,w_17
1,1e-15,0.0180668,0.0178559,-5.97812,4.3598,2.38185,-2.48476,2.56381,-7.36867,-2.57685,...,0.0390847,-0.0168822,5.26454,-7.57855,7.54041,7.22619,5.04505,2.94285,3.09141,1.79822
2,1e-10,0.0180004,0.0177455,-4.5669,3.17107,2.08508,-0.700424,1.55699,-7.12767,-1.19907,...,0.031908,-0.0124549,4.2927,-6.99137,7.39696,3.87942,3.38101,2.28354,1.76188,1.36775
3,1e-08,0.0179974,0.0177455,-4.56684,3.1712,2.08501,-0.700661,1.55697,-7.12761,-1.19828,...,0.0319053,-0.0124529,4.29282,-6.99077,7.39682,3.87939,3.38115,2.28348,1.76173,1.36773
4,0.0001,0.0179974,0.0177394,-4.56691,3.17107,2.08507,-0.700434,1.55699,-7.12766,-1.199,...,0.0319108,-0.0124581,4.29273,-6.99132,7.39694,3.87942,3.38101,2.28353,1.76186,1.36774
5,0.001,0.0180004,0.0177455,-4.56688,3.17106,2.08506,-0.70045,1.55698,-7.12764,-1.19905,...,0.0319023,-0.0124486,4.29272,-6.99134,7.39692,3.87945,3.38101,2.28352,1.76188,1.36774
6,0.01,0.0180004,0.0177455,-4.5668,3.17093,2.0849,-0.70057,1.55698,-7.12738,-1.19923,...,0.0319067,-0.0124549,4.29282,-6.9913,7.39664,3.8797,3.38089,2.28337,1.76202,1.36773
7,1,0.0180276,0.0177885,-4.55698,3.15686,2.06804,-0.713637,1.55673,-7.10001,-1.22197,...,0.031882,-0.0125644,4.30118,-6.98935,7.36731,3.90754,3.36807,2.26775,1.77799,1.36663
8,5,0.0184445,0.0182669,-4.93311,3.25774,2.00507,-1.03988,1.79012,-6.84365,-1.85726,...,0.0332541,-0.0135297,4.43045,-7.10116,7.26832,4.51323,3.57106,2.27971,2.15748,1.48457
9,10,0.0181575,0.0179479,-4.60083,3.03369,1.8688,-0.925444,1.60439,-6.73428,-1.42906,...,0.0311687,-0.0122814,4.21167,-6.67347,7.11846,4.19282,3.28078,2.09721,1.97587,1.37038
10,20,0.0184808,0.0182056,-4.35214,2.60879,1.69069,-0.60355,1.42059,-6.42725,-1.31223,...,0.0292089,-0.0102481,3.69843,-6.32535,6.94735,3.55272,2.7968,1.87057,1.82452,1.24571


Her bir $\lambda$ değeri için $L1$ regularization kullanarak *LogisticRegression* algoritması kullanılarak sınıflandırma modelini olustur.

In [7]:
i = 0
for l in lambdas:
    clf = LogisticRegression(penalty='l1', C=1/l)
    clf.fit(X_train,y_train)
    
    y_hat_train = clf.predict(X_train)
    mse_train = mean_squared_error(y_train, y_hat_train)
    
    y_hat_test = clf.predict(X_test)
    mse_test = mean_squared_error(y_test, y_hat_test)
    
    coef_matrix_simple.iloc[i,0] = l
    coef_matrix_simple.iloc[i,1] = mse_train
    coef_matrix_simple.iloc[i,2] = mse_test
    coef_matrix_simple.iloc[i,3] = clf.intercept_[0]
    coef_matrix_simple.iloc[i,4:] = clf.coef_
    i += 1
    print('Tamamlanan : % {0}'.format(i/len(lambdas)*100), end = '\r',flush=False)
    #os.system('cls' if os.name=='nt' else 'clear')
coef_matrix_simple

,lambda,MSE_Train,MSE_Test,w_0,w_1,w_2,w_3,w_4,w_5,w_6,...,w_8,w_9,w_10,w_11,w_12,w_13,w_14,w_15,w_16,w_17
1,1e-15,0.0159278,0.0160832,-9.16929,9.23126,-5.62013,-16.8524,13.186,-5.73195,-1.69688,...,0.0455713,-0.0220337,6.43747,-8.0584,7.63316,14.5764,6.82073,22.8572,5.63713,2.92898
2,1e-10,0.0159157,0.016071,-9.24431,9.25252,-5.63277,-16.8504,13.1866,-5.68958,-1.6605,...,0.0456935,-0.0221496,6.46761,-8.06157,7.63519,14.6171,6.82192,22.8673,5.64956,2.92587
3,1e-08,0.0150426,0.0152735,-8.93268,9.20493,-5.62073,-16.8075,13.1605,-5.86089,-1.80101,...,0.0451954,-0.0217213,6.34839,-8.05766,7.62395,14.4422,6.80245,22.8279,5.58883,2.922
4,0.0001,0.0150426,0.0152735,-8.91988,9.18859,-5.60891,-16.8225,13.1711,-5.87198,-1.81081,...,0.0451733,-0.0217062,6.34605,-8.05469,7.62704,14.4303,6.80725,22.8303,5.58887,2.92681
5,0.001,0.0159339,0.0160832,-9.16806,9.25096,-5.64021,-16.8454,13.1852,-5.72635,-1.69149,...,0.0455538,-0.0220151,6.43025,-8.06184,7.63307,14.5688,6.81811,22.8769,5.63071,2.929
6,0.01,0.0150517,0.0152551,-8.99675,9.18837,-5.5895,-16.8442,13.1896,-5.82086,-1.7673,...,0.0452805,-0.0217678,6.36087,-8.06185,7.62869,14.4755,6.80768,22.8316,5.6008,2.92857
7,1,0.0160457,0.0161998,-8.92033,6.69925,-2.46276,-16.4913,12.9464,-5.88276,-1.7159,...,0.0451099,-0.0217078,6.38256,-7.98918,7.60503,14.3323,6.96888,19.4339,5.53196,2.90212
8,5,0.0162451,0.0163777,-8.6645,4.63537,0,-14.1502,11.2566,-5.93151,-1.57587,...,0.0441638,-0.0210219,6.27113,-7.81644,7.53604,13.612,6.88195,14.6604,5.30537,2.45458
9,10,0.0165139,0.0165433,-8.40124,4.28698,0,-11.2712,9.29767,-5.94246,-1.43013,...,0.0430888,-0.0202479,6.12518,-7.65003,7.46652,12.7305,6.58521,11.8091,5.0232,1.76679
10,20,0.0174264,0.0172732,-7.79002,4.00023,0,-6.37931,6.21933,-6.03558,-1.23793,...,0.0409913,-0.0187352,5.80205,-7.37948,7.34737,11.0107,6.07489,7.14805,4.43905,0.40585


Pandas dataframe ile **SQL** benzeri sorgular yapabilirsiniz.

In [8]:
coef_matrix_simple.query("w_2 == 0 and w_3 == 0")

,lambda,MSE_Train,MSE_Test,w_0,w_1,w_2,w_3,w_4,w_5,w_6,...,w_8,w_9,w_10,w_11,w_12,w_13,w_14,w_15,w_16,w_17
11,50,0.0182088,0.0179909,-6.17321,3.44195,0,0,1.22388,-6.01763,-0.767175,...,0.0346822,-0.013928,4.6388,-6.87481,7.12158,7.12364,5.02046,1.454,3.04463,0
12,100,0.0189702,0.0188128,-4.43097,2.10607,0,0,0.671523,-5.91351,-0.279651,...,0.0280667,-0.00900139,3.27059,-6.56665,6.93329,3.66843,3.91924,0.499366,2.99922,0
13,1000,0.0292967,0.0290811,0,0.200032,0,0,0,-6.04961,0,...,0.0154422,0.00180823,0,-3.73604,5.5956,0,0.873014,0,0,0
